In [ ]:
from pyspark import SparkContext 
dat=[1,2,3,4,5]
sc=SparkContext("local","wordcount")
disDat=sc.parallelize(dat)
disDat.collect()
disDat=sc.parallelize(dat,10)

In [ ]:
from pyspark import SparkContext
sc = SparkContext( 'local', 'mywordcount')
textFile = sc.textFile("file:///opt/text/python/pyspark/data.txt")
wordCount = textFile.flatMap(lambda line: line.split(" ")).map(lambda word: (word,1)).reduceByKey(lambda a, b : a + b)
wordCount.collect()
wordCount.foreach(print)


# RDD综合应用 

In [ ]:
import random as rd
for i in range(10):
    num=rd.random()*100
    str1=str(i)+ " "+str(int(num))
    print(str1)

### RDD操作计算平均年龄

由年龄文件生成人rdd1

In [ ]:
from pyspark import SparkContext
sc = SparkContext( 'local', 'mywordcount')
rdd1=sc.textFile("age.txt")
rdd1.collect()

RDD转换（年龄，1）形式的RDD

In [ ]:
rdd2 = rdd1.map(lambda line : line.split(" ")).map(lambda x:(int(x[1]),1))
rdd2.collect()

In [ ]:
rdd2.keys().reduce(lambda x,y : x+y)/rdd2.values().count()

In [ ]:
rdd = sc.parallelize([("spark",2),("hadoop",6),("hadoop",4),("spark",6)])
rdd.mapValues(lambda x : (x,1)).reduceByKey(lambda x,y : (x[0]+y[0],x[1] + y[1])).mapValues(lambda x : (x[0] / x[1])).collect()


In [ ]:
rdd=sc.parallelize(["b","a","c"])
rdd.persist()

In [ ]:
rdd.is_cached

In [ ]:
rdd.count()

In [ ]:
rdd.unpersist()
rdd.is_cached

In [ ]:
import pandas as pd
print(pd.__version__)

# 实验四---SparkDataFram 

In [ ]:

from pyspark.sql import Row
from pyspark import SparkContext
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName("rddDataFrame").master("local[*]").getOrCreate()
#读取数据
book=spark.read.text('file:///opt/text/python/pyspark/book.txt')
book.take(5)

In [ ]:
type(book)

In [ ]:
book.printSchema()

In [ ]:
book_rdd=book.rdd.map(lambda x:x[0].split(",")).map(lambda x:Row(id=x[0],name=x[1],rating=x[2],price=x[3],publish=x[4],rul=x[5]))
book_rdd.take(5)

In [ ]:
book_df=spark.createDataFrame(book_rdd)
book_df.show(5)

In [ ]:
book_df.printSchema()

In [ ]:
#指定列输出
book_df.select('id','name').show()

In [ ]:
#查看方法和属性
dir(book_df)

In [ ]:
book_df.select(book_df['id'],book_df['name'],book_df['rating']-1).show()

In [ ]:
import pandas as pd
book_pds=book_df.groupBy('publish').count()
print(book_pds.show())

In [ ]:
book_pds=book_pds.toPandas()
print(book_pds)

In [ ]:
ax=book_pds.head(n=10).plot(x='publish',y=['count'],kind='bar',title='book count of publish')

In [ ]:
from pyspark.sql import Row
from pyspark import SparkContext
from pyspark.sql import SparkSession
import pandas as pd

spark=SparkSession.builder.appName("rddDataFrame").master("local[*]").getOrCreate()
#读取数据
book=spark.read.text('file:///opt/text/python/pyspark/book.txt')
#转化Rdd
book_rdd=book.rdd.map(lambda x:x[0].split(",")).map(lambda x:Row(id=x[0],name=x[1],rating=x[2],price=x[3],publish=x[4],rul=x[5]))
#创建dataFrame
book_df=spark.createDataFrame(book_rdd)
# book_df.select(book_df['id'],book_df['name'],book_df['rating']-1).show()
#pandas
book_pds=book_df.groupBy('publish').count()
book_pds=book_pds.toPandas()
ax=book_pds.head(n=10).plot(x='publish',y=['count'],kind='bar',title='book count of publish')
print(ax)

# 实验五

In [ ]:
from pyspark.sql import Row
from pyspark import SparkContext
from pyspark.sql import SparkSession
import pandas as pd

spark=SparkSession.builder.appName("rddDataFrame").master("local[*]").getOrCreate()
#读取数据
book=spark.read.text('file:///opt/text/python/pyspark/book.txt')

#查看前五行数据
# book.take(5)

#查看类型
# type(book)

#打印表头数据
# book.printSchema()

#将book的DataFrame转化为RRD
book_rdd=book.rdd.map(lambda x:x[0].split(",")).map(lambda  x:Row(id=x[0],name=x[1],rating=x[2],price=x[3],publish=x[4],url=x[5]))
# book_rdd.take(5)
#创建dataFrame
book_df=spark.createDataFrame(book_rdd)
#查询前五行数据
# book_df.show(5)
#查询前50行
#打印schema
# book_df.printSchema()
#创建临时表 
book_df.registerTempTable("tb_book")
#使用sql语句查询 
spark.sql("select * from tb_book").show(15)
#模糊查询书名包含“微积分”的书 
spark.sql("select * from tb_book where name like '%微积分%'").show()
#输出图书的前10行的name和price字段信息
spark.sql("select name,price from tb_book ").show(10)
#统计书名包含“微积分”的书的数量 
spark.sql("select count(*) from tb_book where name like '%微积分%'").show()
#查询评分大于9的图书 
spark.sql('select * from tb_book where rating>9').show(10)
#计算所有书名包含“微积分”的评分平均值 
spark.sql("select avg(rating) rating_avg from tb_book where name like '%微积 分%'")
#把书目按照评分从高到低进行排列 
spark.sql("select id,name,rating from tb_book order by rating desc").show(15)
#把图书按照出版社进行分组 
spark.sql("select publish,count(name) as cnt from tb_book group by publish order by cnt desc").show()

# 实验六

In [ ]:
from pyspark.streaming import StreamingContext
from pyspark import SparkContext

# sc=SparkContext("local","appName")
sc=SparkContext("local","PythonStreamingHdfSWordCount")
scc=StreamingContext(sc,5)

type(scc)

# lines=scc.textFileStream("file:///opt/text/python/listen")
# lines=scc.textFileStream("hdfs://localhost:8020/listen")
lines=scc.socketTextStream('localhost',9999)



counts = lines.flatMap(lambda line:line.split(" ")).map(lambda word:(word,1)).reduceByKey(lambda a,b:a+b)

counts.pprint()
scc.start();scc.awaitTermination()

# 实验七

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

sc = SparkContext()
sc.setLogLevel("WARN")
spark = SparkSession.builder.getOrCreate()
#读取用户
user_df=spark.read.text('file:///opt/text/python/ml-100k/u.user')
user_df.show(10)

#为用户数据添加 schem
from pyspark import Row
user_rdd = user_df.rdd.map(lambda x:x[0].split('|')).map(lambda x:Row(id=x[0],age=x[1],sex=x[2],job=x[3],code=x[4]))
user_rdd.take(5)


#创建用户 dataframe
user_df=spark.createDataFrame(user_rdd)

#注册临时表
user_df.registerTempTable("user")
spark.sql('select id,age,sex,job,code from user').show()


#读取评分数据
rating_df = spark.read.text('file:///opt/text/python/ml-100k/u.data')
rating_df.show(5)


#为评分数据添加 schema
rating_rdd = rating_df.rdd.map(lambda x:x[0].split()).\
    map(lambda x: Row(user_id=x[0], film_id=x[1], rating=x[2], time=x[3]))
rating_rdd.take(5)


#创建评分 dataframe
rating_df = spark.createDataFrame(rating_rdd)

#注册评分临时表
rating_df.registerTempTable('rating')
spark.sql("select user_id,film_id,rating,time from rating").show()


#读取电影信息
film_df = spark.read.text('file:///opt/text/python/ml-100k/u.item')
film_df.show()


#为电影数据添加 schema
film_rdd = film_df.rdd.map(lambda x:x[0].split('|')).map(lambda x:Row(id=x[0],title=x[1],post_time=x[2],url=x[4]))
film_rdd.take(5)


#创建电影 dataframe
film_df = spark.createDataFrame(film_rdd)
#注册电影临时表
film_df.registerTempTable('film')
spark.sql('select * from film').show()

#导入 Spark 机器学习包：协同过滤算法
from pyspark.mllib.recommendation import ALS
#ALS.trainImplicit 中第一个参数 RDD 是固定格式（用户编号，电影编号，评分值）
rdd = rating_rdd.map(lambda x: (x[3], x[0], x[1]))
rdd.take(5)

#训练模型
model = ALS.train(rdd, 50, 10, 0.01)
#使用模型进行推荐，参数为用户 id，推荐个数
user_film = model.recommendProducts(881250949, 3)
print(user_film)

#显示推荐电影名称
type(user_film)
for film in user_film:
    spark.sql("select id ,title,post_time,url , "+str(film[2])+" as similar from film where id="+str(film[1])).show()


#查看用户看过的电影
spark.sql("select a.user_id,b.title from rating a,film b where a.film_id=b.id and a.user_id=112").show()


#保存模型
import os
import shutil
save_path = '/opt/text/python/film_model'
if os.path.exists(save_path):
    shutil.rmtree(save_path)
model.save(sc, save_path)
print('model saved!')
#调用模型
from pyspark.mllib.recommendation import MatrixFactorizationModel
model = MatrixFactorizationModel.load(sc, save_path)
result = model.recommendProducts(881250949, 3)
for film in user_film:
    spark.sql("select id , title,post_time,url , "+str(film[2])+" as similar from film where id="+str(film[1])).show()






# 实验八 

In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
sc=SparkContext("local","texi")
taxi_data=sc.textFile('file:///opt/text/python/taxi.csv')
taxi_rdd=taxi_data.map(lambda line:line.split(','))
from  pyspark.ml.linalg  import  Vectors
taxi_row=taxi_rdd.map(lambda x: (Vectors.dense (x[2],x[1]), ))
sqlsc=SparkSession.builder.getOrCreate()
taxi_df=sqlsc.createDataFrame(taxi_row,["features"])
from  pyspark.ml.clustering  import  KMeans
kmeans=KMeans(k=3, seed=1)  #聚成3类
model=kmeans.fit(taxi_df)    #注意，传入的DataFrame是矢量名称为features的集合
centers=model.clusterCenters()  #产生聚类集合
print(centers)
# Kmeans方法中包含啊若干个参数，k为分簇个数，seed为种子点，上方为核心代码。
# 代码解析如下：
# 第一行：从pyspark.ml.clustering 中导入KMeans。
# 第二行：创建kmeans实例，k为分簇个数，seed为种子点。
# 第三行：调用kmeans.fit方法训练形成模型
# 第四行：产生聚类集合
# 第五行：输出集合
# 聚类的结果是一系列的点集，这些点集也就是出租车聚集的地区，上述代码将数据聚类成3类，如下


[array([104.01762438,  30.65643936]), array([103.6522621 ,  30.89418873]), array([104.08803516,  30.64616188])]
